Mounting google drive for downloading datasets and models


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!rm -r pokemon

In [ ]:
!rm -r drive/My\ Drive/Colab\ Notebooks/GAN/generated

Unzip dataset from google drive


In [ ]:
!unzip drive/My\ Drive/Colab\ Notebooks/GAN/pokemon_expanded.zip

Import libraries

In [ ]:
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import Conv2D, Conv3D, UpSampling2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from scipy.misc import imsave as ims
import matplotlib.pyplot as plt
from PIL import Image
import sys
import os
import numpy as np
from keras.models import model_from_json
import random

Downloading images from folder, '.jpg' format


In [ ]:
def load_images_to_data(image_directory, im_dim):
    features_data = np.zeros((1, im_dim, im_dim, 3))
    list_of_files = os.listdir(image_directory)
    for file in list_of_files:
        image_file_name = os.path.join(image_directory, file)
        if ".jpg" in image_file_name:
            img = Image.open(image_file_name).resize((im_dim, im_dim)).convert("RGB")

            im2arr = np.asarray(img).reshape(1, im_dim, im_dim, 3)
            features_data = np.append(features_data, im2arr, axis=0)
        
    return features_data[1:,: ,: ,:]

Function return generator model

In [ ]:
def build_generator():

    noise_shape = (100,)

    model = Sequential()

    model.add(Dense(128 * 16 * 16, activation="relu", input_shape=noise_shape))
    model.add(Reshape((16, 16, 128)))
    model.add(BatchNormalization(momentum=0.8))
    model.add(UpSampling2D())
    model.add(Conv2D(128, kernel_size=3, padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(UpSampling2D())
    model.add(Conv2D(64, kernel_size=3, padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Conv2D(3, kernel_size=3, padding="same"))
    model.add(Activation("tanh"))

    noise = Input(shape=noise_shape)
    img = model(noise)

    return Model(noise, img)

Function return discriminator model

In [ ]:
def build_discriminator(img_shape):

        model = Sequential()

        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))

        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))

        img = Input(shape=img_shape)
        validity = model(img)

        return Model(img, validity)


Saving images, combine dim*dim images per file

In [ ]:
def save_imgs(gen, epoch, save_folder, dim):
    r, c = dim, dim
    noise = np.random.normal(0, 1, (r * c, 100))
    gen_imgs = gen.predict(noise)

    gen_imgs = 0.5 * gen_imgs + 0.5
    ims(save_folder + 'images/pokemon_' +  epoch + '.png', merge(gen_imgs,[dim, dim]))

In [ ]:
def merge(images, size):
    h, w = images.shape[1], images.shape[2]
    img = np.zeros((h * size[0], w * size[1], 3))

    for idx, image in enumerate(images):
        i = idx % size[1]
        j = idx // size[1]
        img[j*h:j*h+h, i*w:i*w+w, :] = image

    return img

Training models

In [ ]:
def train(gen, dis, comb, epochs, batch_size, save_folder, X_train):
  
    iterations = 0
    save_interval = 200
    
    # Scale to -1, 1
    X_train = (X_train.astype(np.float32) - 127.5) / 127.5

    half_batch = int(batch_size / 2)

    for epoch in range (epochs + 1):
      for index in range(int(X_train.shape[0]/half_batch)):
          # ---------------------
          #  Train Discriminator
          # ---------------------

#         idx = np.random.randint(0, X_train.shape[0], half_batch)
#         imgs = X_train[idx]
        # Get batch
        imgs = X_train[index * half_batch : (index + 1) * (half_batch)]

        # Generate new images
        noise = np.random.normal(0, 1, (half_batch, 100))
        gen_imgs = gen.predict(noise)

        # Discriminator trainning
        dis_loss_real = dis.train_on_batch(imgs, np.ones((half_batch, 1)))
        dis_loss_fake = dis.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))
        dis_loss = 0.5 * np.add(dis_loss_real, dis_loss_fake)
#           dis_los = self.discriminator.train_on_batch(train_batch, target)

        noise = np.random.normal(0, 1, (batch_size, 100))

        # Generator training
        gen_loss = comb.train_on_batch(noise, np.ones((batch_size, 1)))

        # Logs
#               if epoch % save_interval == 0 & index == 0:
        if iterations % save_interval == 0:
#                 print("iteration %f, epoch %f" % (iterations, epoch))
          print ("epoch: %d iter: %d D loss: %f, acc.: %.2f%% | G loss: %f" % 
                 (epoch, iterations, dis_loss[0], 100*dis_loss[1], gen_loss))

#         if epoch == 0:
        if iterations == 0:
            model_json = gen.to_json()
            with open(save_folder + "weights/generator.json", "w") as json_file:
                json_file.write(model_json)

            model_dis_json = dis.to_json()
            with open(save_folder + "weights/discriminator.json", "w") as json_file:
                json_file.write(model_dis_json)
        
        # Save models & generated images
        if iterations % save_interval == 0:
#         if epoch % save_interval == 0:
            save_imgs(gen, str(epoch), save_folder, 1)
            gen_name = save_folder + "weights/gen_" + str(epoch) + ".h5"
            gen.save_weights(gen_name)

            dis_name = save_folder + "weights/dis_" + str(epoch) + ".h5"
            dis.save_weights(dis_name)

        iterations += 1

Models initialization, return discriminator, generator, combined

In [ ]:
 def init_model():
    img_rows = 64
    img_cols = 64
    channels = 3

    optimizer = Adam(0.0002, 0.5)

    # Build and compile the discriminator
    discriminator = build_discriminator((img_rows, img_cols, channels))
    discriminator.compile(loss='binary_crossentropy',
        optimizer=optimizer,
        metrics=['accuracy'])
#           metrics=['mae'])

    # Build and compile the generator
    generator = build_generator()
    generator.compile(loss='binary_crossentropy', optimizer=optimizer)

    # The generator takes noise as input and generated imgs
    z = Input(shape=(100,))
    img = generator(z)

    # For the combined model we will only train the generator
    discriminator.trainable = False
    
    # The valid takes generated images as input and determines validity
    valid = discriminator(img)

    # The combined model  (stacked generator and discriminator) takes
    # noise as input => generates images => determines validity
    combined = Model(z, valid)
    combined.compile(loss='binary_crossentropy', optimizer=optimizer)
    
    return discriminator, generator, combined

Load model from folder

In [ ]:
def load_model(model_folder, model_name, model_type):
  # load json and create model
    json_file = open(model_folder + 'weights/' + model_type + '.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights(model_folder+ 'weights/' + model_name)#"weights/model.h5")
    print("Loaded model from disk")
    return loaded_model
  

Load pretrained models

In [ ]:
 def load_pretrained_models(models_folder, dis, gen):
    img_rows = 64
    img_cols = 64
    channels = 3

    optimizer = Adam(0.0002, 0.5)

#     models_folder = 'drive/My Drive/Colab Notebooks/GAN/'


    # Build the discriminator
    discriminator = load_model(models_folder, dis, 'discriminator')
    discriminator.compile(loss='binary_crossentropy',
      optimizer=optimizer,
      metrics=['accuracy'])

    # Build a the generator
    generator = load_model(models_folder, gen, 'generator')
    generator.compile(loss='binary_crossentropy', optimizer=optimizer)

    # The generator takes noise as input and generated imgs
    z = Input(shape=(100,))
    img = generator(z)

    # For the combined model we will only train the generator
    discriminator.trainable = False

    
    # The valid takes generated images as input and determines validity
    valid = discriminator(img)

    # The combined model  (stacked generator and discriminator) takes
    # noise as input => generates images => determines validity
    combined = Model(z, valid)
    combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    return discriminator, generator, combined

Create directories

In [ ]:
# save_folder = 'drive/My Drive/Colab Notebooks/GAN/'
save_folder = ''

if not os.path.exists(save_folder + '/images/'):
    os.makedirs(save_folder + '/images/')
    
if not os.path.exists(save_folder + '/generated/'):
    os.makedirs(save_folder + '/generated/')

if not os.path.exists(save_folder + '/weights/'):
    os.makedirs(save_folder + '/weights/')

Load dataset

In [ ]:
data_train = load_images_to_data('pokemon', 64)

Model training

In [ ]:
batchsize = 50
epochs = 200

save_folder = ''
# dis = 'dis_100.h5'
# gen = 'gen_100.h5'
# discriminator, generator, combined = load_pretrained_models(save_folder + 'AUG_100epochs_64/', dis, gen)
discriminator, generator, combined = init_model()
train(generator, discriminator, combined, epochs, batchsize, save_folder, data_train)


Function generating new images

In [ ]:
def generate_image(save_folder, model, dim, name):
    noise = np.random.normal(0, 1, (dim * dim, 100))
    gen_imgs = model.predict(noise)
    # Rescale images 0 - 1
    gen_imgs = 0.5 * gen_imgs + 0.5 
    ims(save_folder + str(name) + '.png', merge(gen_imgs,[dim, dim]))

Output folder for new images

In [ ]:
# save_folder = 'drive/My Drive/Colab Notebooks/GAN/generated/'
save_folder = 'generated/'
model_name = 'gen_28.h5'
model_type = 'generator'


In [ ]:
generator = load_model('GAN_models/EXTENDED_200_EPOCH_64_ALLDATA/', model_name, 'generator')

Generate 10x10 images 

In [ ]:
generate_image(save_folder, generator, 10, 'test')